In [1]:
# Célula 1: Instalar bibliotecas
!pip install -q google-genai google-adk

In [2]:
# Célula 2: Importar e Configurar a API Key
import os
from google.colab import userdata
import google.generativeai as genai # Importa a biblioteca genai

# Carrega a chave da API dos Secrets do Colab e a define como uma variável de ambiente
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("API Key carregada com sucesso dos Secrets do Colab.")

    # Configura a biblioteca genai com a API Key
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Biblioteca google.generativeai configurada com a API Key.")

except Exception as e:
    print(f"Erro ao carregar ou configurar a API Key: {e}")
    print("Por favor, verifique se a API Key está salva corretamente como 'GOOGLE_API_KEY' nos Secrets do Colab.")

# Você pode definir o ID do modelo aqui também, se preferir, ou na próxima célula
MODEL_ID = "gemini-2.0-flash" # Verifique se este é o ID do modelo que você quer usar
print(f"Modelo Gemini a ser usado: {MODEL_ID}")

API Key carregada com sucesso dos Secrets do Colab.
Biblioteca google.generativeai configurada com a API Key.
Modelo Gemini a ser usado: gemini-2.0-flash


In [3]:
# Célula 3: Importar componentes do ADK e outras bibliotecas
from google.adk.agents import Agent
from google.adk.runners import Runner
# Célula 3: Importar componentes do ADK e outras bibliotecas
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search # Importa a ferramenta de busca
from google.adk.tools import built_in_code_execution
# from google.adk.tools import Maps # Se existir no ADK e você for usar, importe aqui
# from google.adk.tools import Google Flights # Se existir no ADK e você for usar, importe aqui

from google.genai import types  # Tipos de dados para interagir com o modelo (Content, Part)
from datetime import date, timedelta # Para trabalhar com datas
import textwrap # Para formatar texto
from IPython.display import display, Markdown # Para exibir Markdown no Colab
import requests # Geralmente útil para requisições web, talvez usado internamente pelo ADK ou ferramentas
import warnings # Para controlar warnings

warnings.filterwarnings("ignore") # Opcional: ignora algumas mensagens de warning

print("Componentes ADK e outras bibliotecas importadas com sucesso.")

from google.genai import types  # Tipos de dados para interagir com o modelo (Content, Part)
from datetime import date, timedelta # Para trabalhar com datas
import textwrap # Para formatar texto
from IPython.display import display, Markdown # Para exibir Markdown no Colab
import requests # Geralmente útil para requisições web, talvez usado internamente pelo ADK ou ferramentas
import warnings # Para controlar warnings

warnings.filterwarnings("ignore") # Opcional: ignora algumas mensagens de warning

print("Componentes ADK e outras bibliotecas importadas com sucesso.")

Componentes ADK e outras bibliotecas importadas com sucesso.
Componentes ADK e outras bibliotecas importadas com sucesso.


In [4]:
# Célula 4: Função Auxiliar para Chamar o Agente (call_agent) - Com Tratamento de Erro Melhorado
import time # Importa time para time.sleep()
# Garanta que as imports abaixo estejam em células ANTES desta ou no topo do notebook
# from google.adk.agents import Agent
# from google.adk.runners import Runner
# from google.adk.sessions import InMemorySessionService
# from google.genai import types
from google.genai.errors import ClientError # <<< Importa o erro específico da API

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    max_retries = 3 # Número máximo de tentativas em caso de erro retryable (original + 2 retentativas)
    base_delay = 5 # Atraso base em segundos antes de tentar novamente (5s, depois 10s, 20s...)

    for attempt in range(max_retries):
        try:
            print(f"DEBUG call_agent: Tentativa {attempt + 1}/{max_retries} para rodar o agente para a mensagem: '{message_text[:50]}...'") # Print de debug
            # Itera assincronamente pelos eventos retornados durante a execução do agente
            # Removemos request_options pois causou erro anteriormente
            for event in runner.run(user_id="user1", session_id="session1", new_message=content):
                if event.is_final_response():
                    for part in event.content.parts:
                        if part.text is not None:
                            final_response += part.text
                            final_response += "\n"
            # Se chegou aqui, a chamada foi bem sucedida, sai do loop de retries
            print(f"DEBUG call_agent: Agente rodou com sucesso na tentativa {attempt + 1}.")
            return final_response

        except ClientError as e:
            # Captura erros específicos da API do GenAI (status code 4xx, 5xx)
            print(f"DEBUG call_agent: ClientError capturado na tentativa {attempt + 1}: Status {e.status_code}, Mensagem: {e.message}")
            if e.status_code == 429 and attempt < max_retries - 1:
                # É um erro de quota e ainda temos tentativas restantes
                retry_delay = base_delay * (2 ** attempt) # Backoff exponencial simples (5s, 10s, 20s...)
                print(f"DEBUG call_agent: Erro 429 (Quota Excedida). Tentando novamente em {retry_delay} segundos (Tentativa {attempt + 2}/{max_retries})...")
                time.sleep(retry_delay)
                # Continua para a próxima tentativa no loop 'for attempt'
            else:
                # Não é um erro 429, ou não temos mais tentativas, re-lança o erro ClientError
                print(f"DEBUG call_agent: ClientError não retryable ({e.status_code}) ou sem mais tentativas. Re-lançando o erro.")
                raise e # Re-lança o erro ClientError

        except Exception as e:
            # Captura quaisquer outros erros inesperados durante a execução do Runner/Agente
            print(f"DEBUG call_agent: Outro tipo de erro inesperado capturado na tentativa {attempt + 1}: {e}")
            # Para outros erros, não tentamos novamente, re-lança
            raise e # Re-lança o erro genérico

    # Este ponto só seria alcançado se o loop de retries terminasse sem sucesso
    # e a última exceção não fosse re-lançada. O 'raise e' dentro do except já garante
    # que a exceção será propagada. Adicionamos um retorno de fallback caso algo
    # inesperado aconteça e a exceção não seja re-lançada.
    print("DEBUG call_agent: Loop de retries terminou sem sucesso. Um erro deveria ter sido re-lançado.")
    return "ERRO_FATAL_CALL_AGENT: Todas as tentativas falharam."


print("Função call_agent definida (com tratamento de erro aprimorado e retentativas para 429).")

Função call_agent definida (com tratamento de erro aprimorado e retentativas para 429).


In [5]:
# Célula 5: Função Auxiliar para Formatar Markdown (to_markdown)
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  # Substitui bullet points comuns por formato Markdown
  text = text.replace('•', '  *')
  # Adiciona identação de bloco de citação e retorna objeto Markdown
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

print("Função to_markdown definida.")

Função to_markdown definida.


In [6]:
# Célula 6: Definição do Agente Buscador (agente_buscador) - Refinamento das Mensagens de Erro
##########################################
# --- Agente 1: Buscador de Roteiros --- #
##########################################

def agente_buscador (topico, numero_dias, dia, data_formatada, destino, tipo_viagem, lugares_sugeridos_total=[]):
    if lugares_sugeridos_total:
        lugares_str_para_agente = "\n" + "\n".join([f"* {local}" for local in lugares_sugeridos_total])
    else:
        lugares_str_para_agente = "Nenhum local sugerido ainda."

    buscador = Agent(
        name="buscador_de_roteiros",
        model=MODEL_ID,
        instruction=f"""
        Você é um assistente de pesquisa de roteiros especializado em viagens, focado em criar sugestões detalhadas e **ÚNICAS** para **um único dia específico** de uma viagem maior.

        Sua tarefa é utilizar a ferramenta 'Google Search' para encontrar informações relevantes, usando as informações que você recebe na ENTRADA e a LISTA DE LUGARES JÁ SUGERIDOS.

        A ENTRADA conterá todos os detalhes necessários para o dia (Destino, Data específica, etc.) e a LISTA DE LUGARES JÁ SUGERIDOS (apresentada como uma lista com bullet points).

        Leia a LISTA DE LUGARES JÁ SUGERIDOS com atenção e **EVITE ABSOLUTAMENTE sugerir atividades ou locais que JÁ TENHAM SIDO MENCIONADOS NESTA LISTA**. Foque em apresentar opções novas e variadas para o dia atual, explorando diferentes áreas ou temas relacionados ao destino e tipo de viagem.

        Sua missão é gerar sugestões de atividades, locais para visitar, opções de refeição (com bom custo benefício se possível) e ideias de rotas ou como se mover, **focando APENAS NO DIA e DESTINO mencionados na sua ENTRADA**.

        Apresente as sugestões de forma clara e organizada para o dia solicitado, **sem pedir mais informações que já foram fornecidas na ENtrada**. Se a busca não retornar informações suficientes ou opções novas, indique isso.

        Use bullet points ou tópicos curtos para o roteiro diário. Não invente informações. Baseie-se nos resultados da busca e na necessidade de VARIAR em relação à lista de lugares.

        Formato de resposta esperado para o DIA da sua ENTRADA:
        Manhã: Sugestão 1 baseada na busca, como chegar.
        Tarde: Sugestão 2 baseada na busca, opção de almoço.
        Noite: Sugestão 3 baseada na busca, opção de jantar.
        Transporte: Como se locomover neste dia com base na busca.

        Após o roteiro diário, inclua UMA ÚNICA LINHA no final da sua resposta começando EXATAMENTE com "[LOCAIS SUGERIDOS NESTE DIA:] " seguida por uma lista curta e separada por vírgulas dos 3 a 5 PRINCIPAIS locais ou atividades que você sugeriu *neste dia específico*. Esta lista é para o sistema.

        Se não encontrar sugestões específicas na busca para o dia/destino ou opções variadas em relação à lista de lugares, indique isso no roteiro e liste "[LOCAIS SUGERIDOS NESTE DIA:] Nenhum local encontrado" no final.
        """,
        description = "Agente buscador de roteiros diários no Google com memória de locais.",
        tools=[google_search]
    )

    entrada_do_agente_buscador_de_roteiros = f"""
    Por favor, crie sugestões de roteiro detalhadas para o dia com base nestas informações. EVITE SUGERIR LOCAIS DA LISTA ABAIXO:
    DESTINO: {destino}
    DATA ESPECÍFICA DO DIA: {data_formatada}
    DIA DA VIAGEM (numeração): Dia {dia+1} (de {numero_dias} dias totais)
    TIPO DE VIAGEM: {tipo_viagem}

    # --- LISTA DE LUGARES JÁ SUGERIDOS NA VIAGEM ---
    {lugares_str_para_agente}
    # ----------------------------------------------

    Gere as sugestões focando apenas neste dia específico e inclua a lista de locais sugeridos NESTE DIA no final, conforme instruído.
    """

    # --- Bloco TRY...EXCEPT aprimorado na chamada call_agent está na Célula 4 ---
    # --- Processamento do resultado retornado por call_agent na função agente_buscador ---
    resultados = "Não foi possível gerar sugestões para este dia." # Mensagem padrão caso falhe
    try:
        # call_agent pode retornar a resposta bem-sucedida, um erro ClientError re-lançado,
        # outro Exception re-lançado, ou a string "ERRO_EXECUCAO_AGENTE: ..." em caso de falha interna no runner
        sugestoes_obtidas = call_agent(buscador, entrada_do_agente_buscador_de_roteiros)

        # Verifica o que foi retornado por call_agent
        if sugestoes_obtidas and sugestoes_obtidas.strip(): # Checa se a string não é vazia/só espaços
             if sugestoes_obtidas.startswith("ERRO_EXECUCAO_AGENTE:"):
                  # Se call_agent retornou nossa string de erro customizada
                  mensagem_erro_detalhada = sugestoes_obtidas[len("ERRO_EXECUCAO_AGENTE:"):].strip()
                  resultados = f"Não foi possível obter sugestões para este dia. Ocorreu um erro durante a execução do agente: {mensagem_erro_detalhada}"
             else:
                  # Assume que é uma resposta bem-sucedida (mesmo que possa não ter seguido o formato,
                  # a extração na Célula 8 lida com isso)
                  resultados = sugestoes_obtidas

        else:
             # Se a string retornada por call_agent era vazia ou None
             resultados = "Não foi possível obter sugestões para este dia. O agente não gerou uma resposta válida."

    except ClientError as e:
         # Captura erros específicos da API que foram re-lançados por call_agent
         # (Call_agent tenta retry em 429, outros erros 4xx/5xx chegam aqui)
         print(f"\n--- ERRO DA API PARA O DIA {dia + 1} ({data_formatada}) ---")
         print(f"Detalhe do erro: Status {e.status_code}, Mensagem: {e.message}")
         print("-------------------------------------------------")
         resultados = f"Não foi possível obter sugestões para este dia devido a um erro na comunicação com a API (Status {e.status_code}). Detalhes: {e.message}"


    except Exception as e:
        # Captura quaisquer outros erros Python que possam ter acontecido
        # (Call_agent re-lança outros Exceptions, ou algo pode acontecer aqui mesmo)
        print(f"\n--- ERRO INESPERADO PARA O DIA {dia + 1} ({data_formatada}) ---")
        print(f"Detalhe do erro: {e}")
        print("-------------------------------------------------")
        resultados = f"Não foi possível obter sugestões para este dia devido a um erro inesperado: {e}"


    return resultados

In [7]:
# Nova Célula 7: Coleta de Informações da Viagem

print("--- Bem-vindo ao seu Agente de Viagens Virtual! ---")

# --- Parte 1: Coleta de Informações da Viagem ---
print("--- Início da Coleta de Informações ---")
# ---------------------------------

# 1. Perguntar o destino
destino = input("Primeiro, para qual cidade ou país você gostaria de viajar? ")

# 2. Perguntar as datas da viagem
# >>> COLOQUE DATAS PARA UMA VIAGEM CURTA AQUI (2-4 DIAS PARA TESTE) <<<
data_inicio = input("Ótimo! Qual a data de início da sua viagem (no formato dd/mm/aaaa)? ")
data_fim = input("E qual a data de término da sua viagem (no formato dd/mm/aaaa)? ")

# 3. Perguntar o número de viajantes
numero_viajantes_str = input("Quantas pessoas vão viajar (incluindo você)? ")
try:
    numero_viajantes = int(numero_viajantes_str)
    print(f"Certo, {numero_viajantes} viajante(s).")
except ValueError:
    print("Ops! Parece que você não digitou um número válido para a quantidade de viajantes.")
    numero_viajantes = None

# 4. Perguntar sobre o tipo de viagem
tipo_viagem = input("Que tipo de viagem você busca (ex: aventura, relax, cultural, negócios)? ")

print("\n--- Resumo das Informações da sua Viagem ---")
print(f"Destino: {destino}")
print(f"Início: {data_inicio}")
print(f"Término: {data_fim}")
if numero_viajantes is not None: print(f"Número de Viajantes: {numero_viajantes}")
else: print("Número de Viajantes: Informação não fornecida ou inválida")
print(f"Tipo de Viagem: {tipo_viagem}")
print("-------------------------------------------\n")

# Note: Esta célula termina aqui. As variáveis (destino, data_inicio, etc.)
# manterão seus valores para serem usados na próxima célula.

--- Bem-vindo ao seu Agente de Viagens Virtual! ---
--- Início da Coleta de Informações ---
Primeiro, para qual cidade ou país você gostaria de viajar? Amsterdam
Ótimo! Qual a data de início da sua viagem (no formato dd/mm/aaaa)? 10/07/2025
E qual a data de término da sua viagem (no formato dd/mm/aaaa)? 14/07/2025
Quantas pessoas vão viajar (incluindo você)? 2
Certo, 2 viajante(s).
Que tipo de viagem você busca (ex: aventura, relax, cultural, negócios)? passeio

--- Resumo das Informações da sua Viagem ---
Destino: Amsterdam
Início: 10/07/2025
Término: 14/07/2025
Número de Viajantes: 2
Tipo de Viagem: passeio
-------------------------------------------



In [8]:
# Célula 8: Processamento de Datas e Loop Diário - Versão com Memória Sumarizada
import time
import datetime
import textwrap # Garante que textwrap está disponível aqui para to_markdown
from IPython.display import display, Markdown # Garante display e Markdown aqui para to_markdown
import re # Importa a biblioteca de expressões regulares, útil para extração

# Você precisará garantir que a função to_markdown esteja definida em uma célula ANTES desta (Célula 5, na nossa estrutura sugerida)
# Você precisará garantir que a função agente_buscador esteja definida em uma célula ANTES desta (Célula 6, na nossa estrutura sugerida)

# --- Parte 2: Processamento de Datas e Loop Diário ---
data_inicio_obj = None
data_fim_obj = None
numero_dias = 0 # Inicializamos como 0

try:
    # Essas variáveis (data_inicio, data_fim) vêm da Célula 7
    # Assumimos que import datetime está em uma célula anterior que já foi executada
    data_inicio_obj = datetime.datetime.strptime(data_inicio, "%d/%m/%Y").date()
    data_fim_obj = datetime.datetime.strptime(data_fim, "%d/%m/%Y").date()

    duracao_viagem = data_fim_obj - data_inicio_obj
    numero_dias = duracao_viagem.days + 1

    print(f"DEBUG: O número de dias calculado é: {numero_dias}")

    if numero_dias <= 0:
        print("Ops! A data de término parece ser anterior à data de início.")
        numero_dias = 0

except ValueError:
    print("Ops! Formato de data inválido (esperado dd/mm/aaaa).")
    numero_dias = 0

# As variáveis destino, tipo_viagem, numero_viajantes também vêm da Célula 7 e estão disponíveis aqui

if numero_dias > 0:
    print("\n--- Gerando Roteiro Diário com IA e Busca ---")

    # --- INICIALIZA A LISTA DE LUGARES SUGERIDOS TOTAL ---
    lugares_sugeridos_total = []
    # ---------------------------------------------------

    # --- INÍCIO DO LOOP DIÁRIO ---
    for dia in range(numero_dias):
        data_do_dia = data_inicio_obj + datetime.timedelta(days=dia)
        data_formatada = data_do_dia.strftime("%d/%m/%Y")

        print(f"\n--- Processando Sugestões para o Dia {dia + 1} ({data_formatada}) ---")
        print(f"Solicitando ao agente buscador informações para o dia {dia + 1}...")
        # print(f"DEBUG MEMÓRIA: Lugares sugeridos TOTAL ANTES do dia {dia+1}: {lugares_sugeridos_total}") # Debugging memória

        # Chama a função agente_buscador para obter as sugestões para este dia
        # >>> Passamos a lista de lugares sugeridos total como memória <<<
        sugestoes_do_dia = agente_buscador(
            topico=f"Roteiro detalhado para o dia {dia + 1} da viagem",
            numero_dias=numero_dias,
            dia=dia,
            data_formatada=data_formatada,
            destino=destino,
            tipo_viagem=tipo_viagem,
            lugares_sugeridos_total=lugares_sugeridos_total # <<< NOVO parâmetro com a lista
        )

        # --- Extrair locais sugeridos NESTE dia da resposta do agente ---
        locais_sugeridos_neste_dia = []
        sugestoes_para_display = sugestoes_do_dia # Inicialmente, o texto completo

        marcador_inicio = "[LOCAIS SUGERIDOS NESTE DIA:]" # O texto exato do marcador
        if marcador_inicio in sugestoes_do_dia:
            try:
                # Encontra a posição do marcador de início
                pos_inicio_marcador = sugestoes_do_dia.find(marcador_inicio)
                # Pega a parte da string APÓS o marcador
                string_locais_bruta = sugestoes_do_dia[pos_inicio_marcador + len(marcador_inicio):].strip()

                # Remove a seção do marcador e a lista de locais do texto principal para exibir
                sugestoes_para_display = sugestoes_do_dia[:pos_inicio_marcador].strip()

                if string_locais_bruta and string_locais_bruta.lower() != "nenhum local encontrado":
                    # Divide a string de locais por vírgulas
                    lista_locais_sujos = string_locais_bruta.split(',')
                    # Limpa espaços em branco de cada item e adiciona à lista para memória
                    locais_sugeridos_neste_dia = [local.strip() for local in lista_locais_sujos if local.strip()]
                    # print(f"DEBUG EXTRAÇÃO: Lista de locais extraída para o dia {dia+1}: {locais_sugeridos_neste_dia}") # Debugging extração
                # else:
                    # print(f"DEBUG EXTRAÇÃO: Marcador encontrado, mas lista de locais vazia ou 'nenhum local encontrado' para o dia {dia+1}.") # Debugging extração

            except Exception as e:
                 print(f"AVISO: Erro ao extrair locais sugeridos do texto do agente para o dia {dia+1}: {e}")
                 # Em caso de erro na extração, mantém o texto completo e a lista de locais sugeridos neste dia vazia

        # --- Atualiza a lista total de locais sugeridos ---
        # Adiciona os locais sugeridos NESTE dia (extraídos acima) à lista total
        # Usamos .extend() para adicionar todos os itens da lista extraída
        lugares_sugeridos_total.extend(locais_sugeridos_neste_dia)

        # Opcional: remover duplicados da lista total, caso o agente repita ou a extração não seja perfeita
        # Isso garante que a lista de "locais a evitar" seja única
        lugares_sugeridos_total = list(set(lugares_sugeridos_total))

        # print(f"DEBUG MEMÓRIA: Lugares sugeridos TOTAL após o dia {dia+1}: {lugares_sugeridos_total}") # Debugging memória


        # Exibir o resultado retornado pelo agente buscador usando display e to_markdown
        print(f"\n--- Sugestões do Agente Buscador para o Dia {dia + 1} ({data_formatada}) ---")
        # >>> Usamos a sugestao_para_display limpa <<<
        display(to_markdown(sugestoes_para_display))
        print("----------------------------------------------------")

        # --- ADICIONA UMA PAUSA PARA EVITAR PROBLEMAS DE QUOTA (429) ---
        if dia < numero_dias - 1:
            print("Aguardando 5 segundos antes de processar o próximo dia...")
            time.sleep(5)
        # ----------------------------------------------------------

    print("\n--- Processamento do Roteiro Diário Concluído ---")
    print(f"Lista final de locais únicos sugeridos na viagem: {lugares_sugeridos_total}") # Opcional: imprimir a lista final
    print("Verifique as sugestões geradas para cada dia acima.")


else:
    print("\nNão foi possível gerar o roteiro devido a datas inválidas ou duração <= 0.")

DEBUG: O número de dias calculado é: 5

--- Gerando Roteiro Diário com IA e Busca ---

--- Processando Sugestões para o Dia 1 (10/07/2025) ---
Solicitando ao agente buscador informações para o dia 1...
DEBUG call_agent: Tentativa 1/3 para rodar o agente para a mensagem: '
    Por favor, crie sugestões de roteiro detalhad...'
DEBUG call_agent: Agente rodou com sucesso na tentativa 1.

--- Sugestões do Agente Buscador para o Dia 1 (10/07/2025) ---


> Com base nas buscas, aqui está uma sugestão de roteiro para o dia 1 em Amsterdam, 10 de julho de 2025:
> 
> **Manhã:**
> 
> *   **Comece com um passeio de barco pelos canais de Amsterdam.** Esta é uma forma relaxante e pitoresca de conhecer a cidade, passando por muitos pontos turísticos importantes. Várias empresas oferecem passeios, com opções de áudio em diversos idiomas. Os passeios geralmente partem da Estação Central.
> *   **Alternativa:** Alugue uma bicicleta e explore a cidade pedalando. Amsterdam é conhecida por suas ciclovias.
> 
> **Tarde:**
> 
> *   **Visite o Bloemenmarkt**, o famoso mercado de flores flutuante. Mesmo que não compre nada, é um lugar vibrante e colorido para passear e apreciar as tulipas (se a estação permitir) e outras flores.
> *   **Almoço:** Experimente o Maoz para um falafel acessível e saboroso, ou procure por opções de "kapsalon" em restaurantes turcos perto da Estação Central.
> 
> **Noite:**
> 
> *   **Explore o bairro Jordaan.** Caminhe pelas ruas charmosas, admire as casas e aproveite a atmosfera tranquila. Há muitos cafés e restaurantes acolhedores na área.
> *   **Jantar:** Considere o Foodhallen para experimentar diferentes comidas locais em um ambiente animado.
> 
> **Transporte:**
> 
> *   Amsterdam tem um excelente sistema de transporte público, incluindo bondes (trams), ônibus e metrô. Para o dia, um bilhete de 24 horas pode ser uma boa opção. Andar de bicicleta também é uma ótima maneira de se locomover.

----------------------------------------------------
Aguardando 5 segundos antes de processar o próximo dia...

--- Processando Sugestões para o Dia 2 (11/07/2025) ---
Solicitando ao agente buscador informações para o dia 2...
DEBUG call_agent: Tentativa 1/3 para rodar o agente para a mensagem: '
    Por favor, crie sugestões de roteiro detalhad...'
DEBUG call_agent: Agente rodou com sucesso na tentativa 1.

--- Sugestões do Agente Buscador para o Dia 2 (11/07/2025) ---


> Aqui estão algumas sugestões de roteiro para o seu segundo dia em Amsterdam, 11 de julho de 2025, evitando os locais já visitados:
> 
> **Manhã:**
> 
> *   **Visite o Rijksmuseum:** Explore a vasta coleção de arte e história holandesa, incluindo obras de Rembrandt e Vermeer. Para chegar lá, você pode pegar o bonde (tram) 2 ou 5 a partir da Estação Central de Amsterdam.
> *   **Opção alternativa:** Se preferir algo menos tradicional, visite o Museu Van Loon, uma casa de canal bem preservada que oferece um vislumbre da vida da aristocracia de Amsterdam.
> 
> **Tarde:**
> 
> *   **Passeie pelo Vondelpark:** Relaxe e aproveite a natureza no maior parque de Amsterdam. Alugue uma bicicleta ou simplesmente caminhe pelos caminhos sinuosos.
> *   **Almoço:** Experimente um "broodje" (sanduíche) em uma das muitas padarias artesanais ao redor do Vondelpark. Uma opção é a "Le Fournil de Sébastien", com pães e doces deliciosos.
> *   **Visite o Museu da Resistência (Verzetsmuseum):** Explore a história da resistência holandesa durante a Segunda Guerra Mundial.
> 
> **Noite:**
> 
> *   **Explore o bairro De Pijp:** Descubra este bairro vibrante e multicultural, com suas ruas estreitas, mercados movimentados e uma grande variedade de restaurantes.
> *   **Jantar:** Desfrute de uma refeição no Albert Cuyp Market, um mercado de rua com diversas opções de comida, ou experimente um dos muitos restaurantes étnicos em De Pijp. O restaurante Bazar oferece culinária do Oriente Médio em um ambiente animado.
> *   **Opção alternativa:** Visite o Heineken Experience para uma experiência interativa sobre a história da cerveja Heineken.
> 
> **Transporte:**
> 
> *   A melhor forma de se locomover em Amsterdam é de bicicleta ou transporte público (tram, ônibus e metrô). Considere comprar um Amsterdam Travel Ticket para uso ilimitado do transporte público. Caminhar também é uma ótima opção, especialmente para explorar os bairros centrais.

----------------------------------------------------
Aguardando 5 segundos antes de processar o próximo dia...

--- Processando Sugestões para o Dia 3 (12/07/2025) ---
Solicitando ao agente buscador informações para o dia 3...
DEBUG call_agent: Tentativa 1/3 para rodar o agente para a mensagem: '
    Por favor, crie sugestões de roteiro detalhad...'
DEBUG call_agent: Agente rodou com sucesso na tentativa 1.

--- Sugestões do Agente Buscador para o Dia 3 (12/07/2025) ---


...

----------------------------------------------------
Aguardando 5 segundos antes de processar o próximo dia...

--- Processando Sugestões para o Dia 4 (13/07/2025) ---
Solicitando ao agente buscador informações para o dia 4...
DEBUG call_agent: Tentativa 1/3 para rodar o agente para a mensagem: '
    Por favor, crie sugestões de roteiro detalhad...'
DEBUG call_agent: Agente rodou com sucesso na tentativa 1.

--- Sugestões do Agente Buscador para o Dia 4 (13/07/2025) ---


> Com base nas informações disponíveis e evitando os locais já visitados, aqui estão algumas sugestões para o seu dia 4 em Amsterdã:
> 
> *   **Manhã:** Explore Amsterdam Noord. Pegue a balsa gratuita atrás da Estação Central para Amsterdam Noord. Visite o A'DAM Lookout para uma vista panorâmica da cidade ou o Eye Filmmuseum para explorar a história do cinema. Para chegar lá, pegue a balsa gratuita para NDSM Werf atrás da Estação Central.
> *   **Tarde:** Descubra Westergas. Localizado no Westerpark, este antigo complexo industrial foi transformado em um centro cultural vibrante. Desfrute de bares, restaurantes e um cinema de arte. Almoce no Westerpark, onde há várias opções de restaurantes e cafés. Para chegar, pegue o ônibus ou bonde até Westerpark.
> *   **Noite:** Jante em um restaurante local. Experimente a culinária italiana no Lo Stivale D'oro, perto da Rembrandtplein, conhecido pelo bom custo-benefício. Para chegar, use o transporte público até Rembrandtplein.
> 
> **Transporte:** Utilize o eficiente sistema de transporte público de Amsterdã, que inclui bondes, ônibus e metrô. Considere adquirir um bilhete diário ou de vários dias da GVB para acesso ilimitado.

----------------------------------------------------
Aguardando 5 segundos antes de processar o próximo dia...

--- Processando Sugestões para o Dia 5 (14/07/2025) ---
Solicitando ao agente buscador informações para o dia 5...
DEBUG call_agent: Tentativa 1/3 para rodar o agente para a mensagem: '
    Por favor, crie sugestões de roteiro detalhad...'
DEBUG call_agent: Agente rodou com sucesso na tentativa 1.

--- Sugestões do Agente Buscador para o Dia 5 (14/07/2025) ---


> Para o dia 14 de julho de 2025 em Amsterdã, aqui estão algumas sugestões de roteiro, evitando os locais já visitados:
> 
> **Manhã:**
> 
> *   **Visite o Museu da Ciência NEMO:** Explore exposições interativas sobre ciência e tecnologia, ótimo para todas as idades. Para chegar, caminhe desde a Estação Central ou pegue o ônibus 22.
> *   **Alternativa:** Se preferir algo diferente, experimente o The Upside Down Amsterdam, um museu interativo dedicado ao Instagram, onde você pode tirar fotos divertidas e criativas.
> 
> **Tarde:**
> 
> *   **Explore o Mercado Ten Katemarkt:** Um mercado local vibrante com uma variedade de produtos, desde alimentos até roupas. É uma ótima opção para almoçar algo rápido e saboroso, como um stroopwafel ou arenque. O mercado fica na Ten Katestraat, em Amsterdam Oud-West.
> *   **Opção de Almoço:** Experimente o Hap Hmm, um restaurante tradicional holandês com pratos locais a preços acessíveis (se estiver aberto para almoço, pois fecha cedo).
> 
> **Noite:**
> 
> *   **Jantar no Sefa Grill Restaurante:** Desfrute de comida turca com grelhados deliciosos e custo-benefício ótimo.
> *   **Passeio pela Biblioteca Cuypers:** Visite a maior e mais antiga biblioteca de história da arte na Holanda.
> 
> **Transporte:**
> 
> *   Utilize o transporte público (ônibus, bondes) ou alugue uma bicicleta para se locomover pela cidade. Caminhar também é uma ótima maneira de explorar as áreas centrais.

----------------------------------------------------

--- Processamento do Roteiro Diário Concluído ---
Lista final de locais únicos sugeridos na viagem: ["Lo Stivale D'oro", 'Bairro Jordaan', 'Vondelpark', 'Museu da Ciência NEMO', 'Foodhallen', 'Bloemenmarkt', 'Oosterpark', 'Maoz', 'Amsterdam Noord', 'Sefa Grill Restaurante', 'Mercado Ten Katemarkt', 'Amsterdam Oost', 'Biblioteca Cuypers', 'Passeio de Barco nos Canais', 'NDSM Wharf', 'Westergas', 'Dappermarkt', 'De Pijp', 'Rijksmuseum', 'Museu Stedelijk']
Verifique as sugestões geradas para cada dia acima.
